In [1]:
import boto3
from requests_aws4auth import AWS4Auth
import json
from elasticsearch import Elasticsearch, RequestsHttpConnection
import time
import datetime
import random
import uuid

In [2]:
session = boto3.Session()

credentials = session.get_credentials()
es_auth = AWS4Auth(credentials.access_key, credentials.secret_key, 'us-east-1', 'es', session_token=credentials.token)

es_host = '' # the Amazon ES domain, including https://
es_headers = { "Content-Type": "application/json" }


es = Elasticsearch([es_host], http_auth=es_auth, use_ssl=True, verify_certs=True, connection_class=RequestsHttpConnection)


In [3]:
usermapping={'mappings': {'_doc': {'properties': {'attachments': {'properties': {'counts': {'properties': {'banRecieved': {'type': 'integer'},
        'karma': {'type': 'integer'},
        'locationFollowing': {'type': 'integer'},
        'post': {'type': 'integer'},
        'reply': {'type': 'integer'},
        'reportRecieved': {'type': 'integer'},
        'reportTo': {'type': 'integer'},
        'share': {'type': 'integer'},
        'block': {'type': 'integer'},
        'friend': {'type': 'integer'},
        'trophy': {'type': 'integer'},
        'vote': {'type': 'integer'}}},
      'profile': {'properties': {
          'birthday': {'type': 'date'},
        'biography': {'type': 'text'},
        'name': {'type': 'text'},
        'imageKey': {'type': 'keyword'},
        'permanentLocation': {'type': 'geo_point'},
        'notifications': {'properties': {
          'notificationFrequency': {'type': 'keyword'},
            'friendPost': {'type': 'boolean'},
            'replyPost': {'type': 'boolean'},
            'trendingActivity': {'type': 'boolean'},
        }}}}}},
#     'marks': {'properties': {'active': {'type': 'date'},
#       'engagement': {'type': 'date'},
#       'post': {'type': 'date'},
#       'referral': {'type': 'date'},
#       'registration': {'type': 'date'},
#       'reply': {'type': 'date'},
#       'report': {'type': 'date'},
#       'share': {'type': 'date'},
#       'vote': {'type': 'date'}}},
    'properties': {'properties': {'accessCode': {'type': 'keyword'},
      'banActive': {'type': 'boolean'},
      'banEnd': {'type': 'date', 'format': 'epoch_millis', 'fields': {
            'long': { 
              'type': 'long'
            }
          }},
      'createdAt': {'type': 'date', 'format': 'epoch_millis', 'fields': {
            'long': { 
              'type': 'long'
            }
          }},
      'countryCode': {'type': 'keyword'},
      'modifiedAt': {'type': 'date', 'format': 'epoch_millis', 'fields': {
            'long': { 
              'type': 'long'
            }
          }},
        'accountActive': {'type': 'boolean'},
      'password': {'type': 'keyword'},
      'username': {'type': 'completion', 'fields': {
            'keyword': { 
              'type': 'keyword'
            },
            'text': { 
              'type': 'keyword'
            },
          }},
      'phoneNumber': {'type': 'keyword'},
      'unverified': {'type': 'boolean'}}},
    'userId': {'type': 'keyword'}}}}}

In [4]:
postmapping={'mappings': {'_doc': {'properties': {
     'attachments': {'properties': {'bucketKey': {'type': 'keyword'},
       'cityName': {'type': 'text'},
       'duration': {'type': 'float'},
       'location': {'type': 'geo_point'},
       'locationAlias': {'type': 'text'},
       'locationName': {'type': 'text'},
        'unitid': {'type': 'keyword'},
       'message': {'type': 'text'},
       'segments': {'type': 'float'}}},
     'children': {'properties': {'attachments': {'properties': {'bucketKey': {'type': 'keyword'},
         'message': {'type': 'text'}}},
       'childId': {'type': 'keyword'},
       'meta': {'properties': {'atCompletion': {'type': 'float'},
         'filesize': {'type': 'long'},
         'format': {'type': 'keyword'},
         'style': {'type': 'keyword'}}},
       'postId': {'type': 'keyword'},
       'type': {'type': 'keyword'}}},
    
     'meta': {'properties': {'duration': {'type': 'float'},
       'filesize': {'type': 'long'},
       'format': {'type': 'keyword'},
       'orientation': {'type': 'keyword'},
       'segments': {'type': 'integer'},
       'style': {'type': 'keyword'},
       'volume': {'type': 'float'}}},
    
     'postId': {'type': 'keyword'},
     
        'accessCode': {'type': 'keyword'},
       'downScore': {'type': 'integer'},
       'replyCount': {'type': 'integer'},
        'replayCount': {'type': 'integer'},
        'viewTime': {'type': 'float'},
       'shareCount': {'type': 'integer'},
       'upScore': {'type': 'integer'},
       'voteScore': {'type': 'integer'},
    'active': {'type': 'boolean'},
     'createdAt': {'type': 'date', 'format': 'epoch_millis', 'fields': {
            'long': { 
              'type': 'long'
            }
          }},
     'eventCode': {'type': 'keyword'},
     'eventType': {'type': 'keyword'},
     'mediaType': {'type': 'keyword'},
     'modifiedAt': {'type': 'date', 'format': 'epoch_millis', 'fields': {
            'long': { 
              'type': 'long'
            }
          }},
     'userId': {'type': 'keyword'}

}}}}

In [5]:
# trackingmapping= {'mappings': {'_doc': { 'properties': {
#                                 'date': {'type': 'date'},
#                                 'location': {'type': 'geo_point'},
#                                 'locationId': {'type': 'keyword'},
#                                 'cellId': {'type': 'keyword'},
#                                 'metrics': {
#                                     'properties': {
#                                         'all': {
#                                             'properties': {
#                                                 'registration': {'type': 'integer'},
#                                                 'engagement': {'type': 'integer'},
#                                                 'post': {'type': 'integer'},
#                                                 'reply':  {'type': 'integer'},
#                                                 'active':  {'type': 'integer'} ,
#                                                 'referral': {'type': 'integer'} ,
#                                                 'share': {'type': 'integer'} ,
#                                                 'report': {'type': 'integer'} ,
#                                                 'vote':  {'type': 'integer'} 
#                                             }
#                                         },
#                                         'daily': {
#                                             'properties': {
#                                                 'registration': {'type': 'integer'},
#                                                 'engagement': {'type': 'integer'},
#                                                 'post': {'type': 'integer'},
#                                                 'reply':  {'type': 'integer'},
#                                                 'active':  {'type': 'integer'} ,
#                                                 'referral': {'type': 'integer'} ,
#                                                 'share': {'type': 'integer'} ,
#                                                 'report': {'type': 'integer'} ,
#                                                 'vote':  {'type': 'integer'} 
#                                             }
#                                         }
#                                     }
#                                 }
#                             }
#                         }
#                     }
#                 }

In [6]:
eventsmapping= {'mappings': {'_doc': {'properties': {'aboutId': {'type': 'keyword'},
     'createdAt': {'type': 'date', 'format': 'epoch_millis', 'fields': {
            'long': { 
              'type': 'long'
            }
          }},
     'eventCode': {'type': 'keyword'},
     'eventType': {'type': 'keyword'},
     'location': {'type': 'geo_point'},
     'mediaType': {'type': 'keyword'},
     'modifiedAt': {'type': 'date', 'format': 'epoch_millis', 'fields': {
            'long': { 
              'type': 'long'
            }
          }},
    'toUserId': {'type': 'keyword'},
     'byUserId': {'type': 'keyword'}}}}}

In [7]:
collegeinfomapping = {'mappings': {'_doc': {'properties': {'active': {'type': 'boolean'},
    'alias': {'type': 'completion'},
    'ccwpath': {'type': 'geo_shape', 'precision': '100.0m'},
    'city': {'type': 'completion', 'fields': {
            'keyword': { 
              'type': 'keyword'
            },
            'text': { 
              'type': 'text'
            },
          }},
    'cover16s': {'type': 'keyword'},
    'fulltime': {'type': 'integer'},
    'headcount': {'type': 'integer'},
    'name': {'type': 'completion', 'fields': {
            'keyword': { 
              'type': 'keyword'
            },
            'text': { 
              'type': 'text'
            },
          }},
    'asciiName': {'type': 'completion', 'fields': {
            'keyword': { 
              'type': 'keyword'
            },
            'text': { 
              'type': 'text'
            },
          }},
    'location': {'type': 'geo_point'},
    'regex': {'type': 'completion'},
    'state': {'type': 'text'},
    'combined': {'type': 'completion'},
    'undergraduate': {'type': 'integer'},
    'unitid': {'type': 'keyword'},
    'countryCode': {'type': 'keyword'},
    'schoolType': {'type': 'keyword'},
    'region': {'type': 'text'},
    'regionAscii': {'type': 'text'},
    'subregion': {'type': 'text'},
    'subregionAscii': {'type': 'text'},
    'timezone': {'type': 'keyword'},
    'urbanization': {'type': 'keyword'},
    'zipcode': {'type': 'keyword'}}}}}

In [13]:
es.indices.delete(index='users')
es.indices.delete(index='posts')
es.indices.delete(index='events')

{'acknowledged': True}

In [8]:
es.indices.create(index='users', body=usermapping)
es.indices.create(index='posts', body=postmapping)
es.indices.create(index='events', body=eventsmapping)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'tracking-location'}

In [ ]:
for i in range(200):
    

In [3]:
es.indices.get(index='events', feature='_mappings')

{'events': {'mappings': {'_doc': {'properties': {'aboutId': {'type': 'keyword'},
     'byUserId': {'type': 'keyword'},
     'createdAt': {'type': 'date',
      'fields': {'long': {'type': 'long'}},
      'format': 'epoch_millis'},
     'eventCode': {'type': 'keyword'},
     'eventType': {'type': 'keyword'},
     'location': {'type': 'geo_point'},
     'mediaType': {'type': 'keyword'},
     'modifiedAt': {'type': 'date',
      'fields': {'long': {'type': 'long'}},
      'format': 'epoch_millis'},
     'toUserId': {'type': 'keyword'}}}}}}

In [ ]:
es.search(index='posts', doc_type='_doc', body={'query': {'bool': {'must': {'match_all': {}}}}})

In [ ]:
es.delete_by_query(index='posts', doc_type='_doc', body={'query': {'bool': {'must': {'match_all': {}}}}})

In [91]:
aa=es.search(index='college-info', doc_type='_doc', body={'query': {'bool': {'must': {'match_all':{}}, 'filter': [
    {'range': {
        'undergraduate':{
            'gte': -1,
            'lte': 1000000
        }
    }}
]
}},'size': 10000})['hits']['hits']

In [92]:
bb = [el for el in aa if el['_source'].get('state')=='California' or el['_source'].get('state')=='New York']

In [98]:
es.create(index='events', doc_type='_doc', id='CUSTOM' + '_' + str(uuid.uuid4()), body={
            'byUserId': str(random.randint(86,86)),
            'createdAt': int(((datetime.datetime.utcnow() - datetime.timedelta(hours=random.randint(-720,720))) - datetime.datetime(1970,1,1)).total_seconds() * 1000),
            'eventCode': 'POST',
            'location': {'lat': 35.299614, 'lon': -120.657310}


        })

{'_index': 'events',
 '_type': '_doc',
 '_id': 'CUSTOM_6238fad4-876b-477e-ba6f-e78adfbb80f4',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 2947,
 '_primary_term': 1}

In [106]:
es.search(index='events', doc_type='_doc', body={
  "_source": false, 
 "query": {
  "bool": {
      "must": {
        "match_all": {}
      },
      "filter": [
          {
          "term": {
            "byUserId": "86"
          }
        }
      ]
    }
  },
  "aggregations" : {
        
        "college-grid" : {
            
            "geohash_grid" : {
                
                "field" : "location",
                "precision" : 4,
                "size": 30
            }
            
        }
    }
}})

GET https://search-pinata-es-wtz7zmneqgtg3g7bypilta3ske.us-east-1.es.amazonaws.com:443/events/_doc/_search [status:400 request:0.075s]


RequestError: RequestError(400, 'illegal_argument_exception', '[geohash_grid] unknown field [filter], parser not found')

In [32]:
random.uniform(0,1)

0.8091990111200967

In [ ]:
for _ in range(10):
    for el in bb:

        event_code = ''
        if random.uniform(0,1) < 0.5:
            event_code='post'
        else:
            event_code='vote'

        es.create(index='events', doc_type='_doc', id=el['_source']['state'] + '_' + str(uuid.uuid4()), body={
            'byUserId': str(random.randint(86,86)),
            'createdAt': int(((datetime.datetime.utcnow() - datetime.timedelta(hours=random.randint(-720,720))) - datetime.datetime(1970,1,1)).total_seconds() * 1000),
            'eventCode': event_code,
            'location': el['_source']['location']


        })

In [52]:
{
  "_source": false, 
 "query": {
  "bool": {
      "must": {
        "match_all": {}
      },
      "filter": [
          {
          "term": {
            "byUserId": "86"
          }
        }
      ]
    }
  },
  "aggregations" : {
        "college-grid" : {
            "geohash_grid" : {
                "field" : "location",
                "precision" : 5,
                "size": 30
            }
        }
    }
}

In [125]:
{
              '_source': False, 
             'query': {
              'bool': {
                  'must': {
                    'match_all': {}
                  },
                  'filter': [
                      {
                      'term': {
                        'byUserId': 3
                      }
                    }
                  ]
                }
              },
              'aggregations' : {
                    'college-grid' : {
                        'geohash_grid' : {
                            'field' : 'location',
                            'precision' : 5,
                            'size': 5-1
                        }
                    }
                }
            }

{'_source': False,
 'query': {'bool': {'must': {'match_all': {}},
   'filter': [{'term': {'byUserId': 3}}]}},
 'aggregations': {'college-grid': {'geohash_grid': {'field': 'location',
    'precision': 5,
    'size': 4}}}}

In [53]:
elochits = es.search(index='events', doc_type='_doc', body=eventlocationquery)

In [56]:
thehits = elochits['aggregations']['college-grid']['buckets']

In [49]:
thisloc

{'lat': 37.6719, 'lon': -122.4585}

In [57]:
thehits[0]

{'key': 'dr5r', 'doc_count': 1763}

In [87]:
genfunctions=[]
if elochits['hits']['total'] > 0:
    for ind, elochit in enumerate(thehits):
        genfunctions.append(
            {
            "linear": {
              "location": {
                "origin": elochit['key'],
                "scale": "0.8km",
                "offset": "4.1km"
              }
            },  
            "weight": elochit['doc_count']
        }
    )
    

In [88]:
genfunctions.append({
            "linear": {
              "location": {
                "origin": thisloc,
                "scale": "25km"
              }
            },  
            "weight": max(thehits, key=lambda x: x['doc_count'])
        })

In [89]:
genfunctions

[{'exp': {'location': {'origin': 'dr5r', 'scale': '25km'}}, 'weight': 1763},
 {'exp': {'location': {'origin': '9q5c', 'scale': '25km'}}, 'weight': 845},
 {'exp': {'location': {'origin': '9qh0', 'scale': '25km'}}, 'weight': 647},
 {'exp': {'location': {'origin': '9q5f', 'scale': '25km'}}, 'weight': 493},
 {'exp': {'location': {'origin': '9mud', 'scale': '25km'}}, 'weight': 373},
 {'exp': {'location': {'origin': 'dr72', 'scale': '25km'}}, 'weight': 364},
 {'exp': {'location': {'origin': '9q9p', 'scale': '25km'}}, 'weight': 335},
 {'exp': {'location': {'origin': '9qh1', 'scale': '25km'}}, 'weight': 319},
 {'exp': {'location': {'origin': '9q8y', 'scale': '25km'}}, 'weight': 307},
 {'exp': {'location': {'origin': 'dr5x', 'scale': '25km'}}, 'weight': 283},
 {'exp': {'location': {'origin': '9mup', 'scale': '25km'}}, 'weight': 282},
 {'exp': {'location': {'origin': '9q5b', 'scale': '25km'}}, 'weight': 268},
 {'exp': {'location': {'origin': '9qh9', 'scale': '25km'}}, 'weight': 258},
 {'exp': {'

In [128]:
{
  'query': {
    'bool': {
      'should': [
        {
          'function_score': {
            'query': {
              'bool': {
                'must':{
                  'match_all': {}
                  }
              }
            },
            'functions': [ {'exp': {'location': {'origin': {'lat': 37.6719, 'lon': -122.4585},
    'scale': '25km'}}}],
            'score_mode': 'avg'
          }
        },
        {
          'bool': {
            'must':{
              'match_all': {}
              }, 
            'filter': {
              'geo_shape': {
              'ccwpath': {
                  'shape': {
                    'type': 'point',
                     'coordinates': [-121.260463,36.871918]
                  },
                  'relation': 'intersects'
                }
              }
            },
            'boost': 2
          }
        }
      ]
    }
  },
'size': 20
}

{'query': {'bool': {'should': [{'function_score': {'query': {'bool': {'must': {'match_all': {}}}},
      'functions': [{'exp': {'location': {'origin': {'lat': 37.6719,
           'lon': -122.4585},
          'scale': '25km'}}}],
      'score_mode': 'avg'}},
    {'bool': {'must': {'match_all': {}},
      'filter': {'geo_shape': {'ccwpath': {'shape': {'type': 'point',
          'coordinates': [-121.260463, 36.871918]},
         'relation': 'intersects'}}},
      'boost': 2}}]}},
 'size': 20}

In [131]:
college_query={
  "query": {
    "bool": {
      "should": [
        {
          "function_score": {
            "query": {
              "bool": {
                "must":{
                  "match_all": {}
                  }
              }
            },
            "functions": [],
            "score_mode": "avg"
          }
        },
        {
          "bool": {
            "must":{
              "match_all": {}
              }, 
            "filter": {
              "geo_shape": {
              "ccwpath": {
                  "shape": {
                    "type": "point",
                     "coordinates": [-121.260463,36.871918]
                  },
                  "relation": "intersects"
                }
              }
            },
            "boost": 2
          }
        }
      ]
    }
  },
'size': 20
}

In [1]:
from eshelper import ESHelper, TemplateCollegePointQuery

In [2]:
esh = ESHelper()

In [3]:
yolo=TemplateCollegePointQuery(esh=esh, 
location={'lat': 37.65642, 'lon': -122.455603}, user_id="46")


In [4]:
import json
json.dumps(yolo.query_college_point())

'{"query": {"bool": {"should": [{"function_score": {"query": {"bool": {"must": {"match_all": {}}}}, "functions": [{"linear": {"location": {"origin": "dr5ru", "scale": "2.45km", "offset": "0.0km", "decay": 0.5}}, "weight": 388}, {"linear": {"location": {"origin": "dr5rh", "scale": "2.45km", "offset": "0.0km", "decay": 0.5}}, "weight": 170}, {"linear": {"location": {"origin": "9q8yy", "scale": "2.45km", "offset": "0.0km", "decay": 0.5}}, "weight": 136}, {"linear": {"location": {"origin": "dr5rv", "scale": "2.45km", "offset": "0.0km", "decay": 0.5}}, "weight": 92}, {"linear": {"location": {"origin": "dr5rs", "scale": "2.45km", "offset": "0.0km", "decay": 0.5}}, "weight": 91}, {"linear": {"location": {"origin": "dr5re", "scale": "2.45km", "offset": "0.0km", "decay": 0.5}}, "weight": 89}, {"linear": {"location": {"origin": "9q9p3", "scale": "2.45km", "offset": "0.0km", "decay": 0.5}}, "weight": 88}, {"linear": {"location": {"origin": "9q5cu", "scale": "2.45km", "offset": "0.0km", "decay": 0

In [5]:
esh.es().search(index='college-info', doc_type='_doc', body=yolo.query_college_point())

{'took': 33,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 11194,
  'max_score': 388.0,
  'hits': [{'_index': 'college-info',
    '_type': '_doc',
    '_id': '476799',
    '_score': 388.0,
    '_source': {'active': True,
     'countryCode': 'US',
     'headcount': 200,
     'fulltime': 132,
     'undergraduate': 200,
     'location': {'lat': 37.65642, 'lon': -122.455603},
     'unitid': '476799',
     'city': 'South San Francisco',
     'state': 'California',
     'zipcode': '94080-1005',
     'urbanization': 'City: Small',
     'region': 'San Mateo County, CA',
     'regionAscii': 'San Mateo County, CA',
     'subregion': 'San Mateo County',
     'subregionAscii': 'San Mateo County',
     'schoolType': 'SCHC',
     'name': 'Unitek College',
     'asciiName': 'Unitek College',
     'regex': ['unitek', 'college'],
     'combined': ['Unitek College'],
     'timezone': 'America/Los_Angeles'}},
   {'_index': 'college-info',
  

SyntaxError: invalid syntax (<ipython-input-6-49cdeed40185>, line 7)

In [6]:
{'_source': False,
     'query': {'bool': {'must': {'match_all': {}},
       'filter': [{'term': {'byUserId':'86'}}]}},
     'aggregations': {'geohash-aggregation': {'geohash_grid': {'field': 'location',
        'precision': 5,
        'size': 5-1}}}}

{'_source': False,
 'query': {'bool': {'must': {'match_all': {}},
   'filter': [{'term': {'byUserId': '86'}}]}},
 'aggregations': {'geohash-aggregation': {'geohash_grid': {'field': 'location',
    'precision': 5,
    'size': 4}}}}

In [86]:
es.search(index='college-info', doc_type='_doc', body=college_query)

{'took': 84,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 11194,
  'max_score': 0.5271499,
  'hits': [{'_index': 'college-info',
    '_type': '_doc',
    '_id': '476799',
    '_score': 0.5271499,
    '_source': {'active': True,
     'countryCode': 'US',
     'headcount': 200,
     'fulltime': 132,
     'undergraduate': 200,
     'location': {'lat': 37.65642, 'lon': -122.455603},
     'unitid': '476799',
     'city': 'South San Francisco',
     'state': 'California',
     'zipcode': '94080-1005',
     'urbanization': 'City: Small',
     'region': 'San Mateo County, CA',
     'regionAscii': 'San Mateo County, CA',
     'subregion': 'San Mateo County',
     'subregionAscii': 'San Mateo County',
     'schoolType': 'SCHC',
     'name': 'Unitek College',
     'asciiName': 'Unitek College',
     'regex': ['unitek', 'college'],
     'combined': ['Unitek College'],
     'timezone': 'America/Los_Angeles'}},
   {'_index': 'college-i

In [5]:
import time
import random
import uuid
import datetime
es.delete_by_query(index='posts', doc_type='_doc', body={'query': {'bool': {'must': {'match_all': {}}}}})

for el in aa:
    for i in range(10,2010,100):
        es.create(index='posts', id=el['_source']['state'] + '_' +  str(i) + '_' + el['_source']['institution'] , doc_type='_doc', body={'createdAt': int(time.time() * 1000), 'voteScore': i, 'active': True, 'attachments': {'location': el['_source']['location']}})
        

In [6]:
for i in range(1000):
    es.create(index='posts', doc_type='_doc', id='bins' + '_' + str(uuid.uuid4()), body={'createdAt': int(time.time() * 1000), 'voteScore': random.randint(-10,300), 'active': True, 'attachments': {'location': {'lat':random.uniform(37.845037,37.898156), 'lon': random.uniform(-122.222900,-122.306757)}}})

In [34]:
for el in aa:
    for i in range(10):
        es.create(index='posts', id='statetime_' + el['_source']['state'] + '_' +  str(i) + '_' + el['_source']['institution'] , doc_type='_doc', body={'createdAt': int(((datetime.datetime.utcnow() - datetime.timedelta(hours=random.randint(-720,720))) - datetime.datetime(1970,1,1)).total_seconds() * 1000), 'voteScore': random.randint(-10,300), 'active': True, 'attachments': {'location': el['_source']['location']}})
        

In [32]:
for i in range(1000):
    es.create(index='posts', doc_type='_doc', id='time' + '_' + str(uuid.uuid4()), body={'createdAt': int(((datetime.datetime.utcnow() - datetime.timedelta(hours=random.randint(-720,720))) - datetime.datetime(1970,1,1)).total_seconds() * 1000), 'voteScore': random.randint(-10,300), 'active': True, 'attachments': {'location': {'lat':random.uniform(37.845037,37.898156), 'lon': random.uniform(-122.222900,-122.306757)}}})

In [29]:
import datetime
import random

In [30]:
int(((datetime.datetime.utcnow() - datetime.timedelta(hours=random.randint(-720,720))) - datetime.datetime(1970,1,1)).total_seconds() * 1000)


1549014597874

In [19]:
type(a)

datetime.datetime

In [82]:
import uuid

In [79]:
random.randint(0,300)

258

In [72]:
random.uniform(-122.222900,-122.306757)

-122.26175368118723

In [73]:
random.uniform(37.845037,37.898156)

37.86899197450625

In [ ]:
es.create(index='posts', id='UCLA10', doc_type='_doc', body={'createdAt': int(time.time() * 1000), 'voteScore': 10, 'location': {'lat': }})